In [72]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import matplotlib.cm as cm
import requests
import json
import pandas as pd
import time
from tqdm import tqdm
import datetime
import os
import sys
import glob
import re
import math
import random
import pickle
import copy
import itertools
import collections
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup

## Invited Speakers

In [7]:
'''
import csv

# Open the input file for reading
with open('AAAI.txt', 'r') as input_file:
    # Create a CSV reader object to read the input file
    reader = csv.reader(input_file, delimiter=',')

    # Loop through each row in the input file
    for row in reader:
        # Extract the year value from the row
        year = row[0]

        # Loop through each name value in the row (starting from index 1)
        for name in row[1:]:
            # Create a new row with the year and name values
            new_row = f"{year},{name}"
            print(new_row)
'''

'\nimport csv\n\n# Open the input file for reading\nwith open(\'AAAI.txt\', \'r\') as input_file:\n    # Create a CSV reader object to read the input file\n    reader = csv.reader(input_file, delimiter=\',\')\n\n    # Loop through each row in the input file\n    for row in reader:\n        # Extract the year value from the row\n        year = row[0]\n\n        # Loop through each name value in the row (starting from index 1)\n        for name in row[1:]:\n            # Create a new row with the year and name values\n            new_row = f"{year},{name}"\n            print(new_row)\n'

## Proceedings

In [75]:
# Initialize
df = pd.DataFrame(columns = ['Full_name',"Year",'Links'])
# create a list from 2000 to 2022 without 2009, 2003 and 2001 and combine that with a list from 80 to 99 without 95, 89, 85 and 81
years = [str(i) for i in range(2003,2023) if i not in [2009, 2003, 2001]]
# years = years + [str(i) for i in range(80,100) if i not in [95, 89, 85, 81]]
pattern = r'<a\s+href="([^"]+)"'
# years

In [76]:
for year in tqdm(years):
    URL_name = "https://dblp.org/db/conf/aaai/aaai" + str(year) + ".html"
    URL = URL_name
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    data = []
    Full_name = []
    Link = []

    results1 = soup.find(id="main")
    results = soup.find(id="main").find_all("ul", class_="publ-list")
    # results1 = results.find(_class="publ-list")
    for row in results:
        ids = [tag['id'] for tag in row.find_all(id=True)]
        for i in ids:
            name_ = results1.find(id=str(i))
            rows = name_.find_all("span")
            for row in rows:
                speaker = row.find_all("a")
                # only take non empty lists
                if speaker:
                    Full_name.append(speaker[0].text)
                    Link.append(re.findall(pattern, str(speaker[0])))

    # create a dataframe
    df1 = pd.DataFrame(columns = ['Full name',"Year",'Links'])
    # append full name to the dataframe
    df1['Full name'] = Full_name
    # append year to the dataframe
    df1['Year'] = year
    df1['Links'] = Link
    time.sleep(1)

    df = df.append(df1, ignore_index=True)

100%|██████████| 18/18 [32:15<00:00, 107.51s/it]


In [78]:
df.Year.value_counts()

2021    8382
2020    8118
2022    7106
2019    5413
2018    4236
2017    2735
2016    2472
2015    2277
2014    1594
2006    1076
2012    1071
2007    1070
2008    1028
2011     978
2005     964
2010     918
2013     762
2004     547
Name: Year, dtype: int64

In [80]:
filepath = "G:/My Drive/Thesis/Data"
df.to_csv(os.path.join(filepath,'Proceedings','AAAI_data.csv'), index=False, encoding='utf-8')